# Runner for Experiment 1


## Experiment

In [1]:
import pandas as pd
import matplotlib.pylab as o

from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.test.mock.backends import FakeManhattan
from qiskit.transpiler.passes import LookaheadSwap, StochasticSwap, SabreSwap
from qiskit.transpiler.passes import DenseLayout, ExVF2Layout, \
                                     NoiseAdaptiveLayout, SabreLayout

from exp.custom_passmanager import custom_pass_manager
from exp.circuit_generators import graphstate_complete, graphstate_ring, graphstate_ring_corners

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/unittest2/compatibility.py:143: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class ChainMap(collections.MutableMapping):


In [2]:
def exp1(circuit, layouter, swapper, backend):
    passmanager = custom_pass_manager(backend, layouter, swapper)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if 'Swap' in kwargs['pass_'].name():
            count_ops_after_map.update(kwargs['dag'].count_ops())
            
    
    transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']
    time = times[layouter.name()] / layouter_reps

    return time, needed_swaps

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Execution

In [3]:
layouter_reps = 4
backend = FakeManhattan()
backend_properties = backend.properties()
coupling_map = CouplingMap(backend.configuration().coupling_map)

#swapper = StochasticSwap(coupling_map, trials=20, seed=0)
swapper = SabreSwap(coupling_map, heuristic="decay", seed=0)

layout_type_dict = {
    'exvf2': ExVF2Layout(coupling_map), 
    'dense': DenseLayout(coupling_map, backend_properties), 
    #'noise_adaptive': NoiseAdaptiveLayout(backend_properties),
    'sabre': SabreLayout(coupling_map, max_iterations=1)
}
circuit_type_dict = {
    'full': graphstate_complete(65),
    '12q_ring': graphstate_ring(12),
    '12q_corner': graphstate_ring_corners(12),
    '12q_full': graphstate_complete(12)
}

In [4]:
df = pd.DataFrame(columns=["layouter", "circuit", "time", "swaps"])

# loop over layouters
for layouter_name, layouter in layout_type_dict.items():

    # loop over circuits
    for circuit_name, circuit in circuit_type_dict.items():

        total_time = 0
        total_swaps = 0
        for rep in range(layouter_reps):
        
            time, swaps = exp1(circuit, layouter, swapper, backend)
            total_time += time
            total_swaps += swaps

        result = {'layouter': layouter_name,
                  'circuit': circuit_name,
                  'time': total_time / layouter_reps,
                  'swaps': total_swaps / layouter_reps}

        df = df.append(result, ignore_index=True)

In [5]:
print(df)

   layouter     circuit       time   swaps
0     exvf2        full   0.006085  3399.0
1     exvf2    12q_ring   0.003150     0.0
2     exvf2  12q_corner   0.003210    10.0
3     exvf2    12q_full  22.278539   101.0
4     dense        full   0.072942  3467.0
5     dense    12q_ring   0.014103     4.0
6     dense  12q_corner   0.015104    10.0
7     dense    12q_full   0.014718   108.0
8     sabre        full   4.042915  3494.0
9     sabre    12q_ring   0.019074    35.0
10    sabre  12q_corner   0.023354    16.0
11    sabre    12q_full   0.062735    76.0
